In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast
import transformers


trainFile="Data/131_yeast_cds_rank.txt"
testFile="Data/test.small.rank.txt"


In [2]:
from transformers import BertTokenizer
from transformers import AutoTokenizer


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:

tokenizer_checkpoint = "zluvolyote/CUBERT_TOKENIZER_Yeast"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

In [5]:
model_checkpoint = "bert-base-uncased"


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [7]:
datasets = load_dataset("text", data_files={"train": trainFile, "validation": testFile})
print(datasets)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])
print(tokenized_datasets)


Using custom data configuration default-3945bbb682633868
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-3945bbb682633868/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 634071
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 12
    })
})
     

#0:   0%|          | 0/159 [00:00<?, ?ba/s]

#2:   0%|          | 0/159 [00:00<?, ?ba/s]

#1:   0%|          | 0/159 [00:00<?, ?ba/s]

#3:   0%|          | 0/159 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (715 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (606 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors


KeyboardInterrupt: 

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# block_size = tokenizer.model_max_length
block_size = 128

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

print(tokenized_datasets)
print(tokenized_datasets['train'][1])

In [ ]:
lm_datasets =tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"CUBERT",
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    weight_decay=0.01,
    num_train_epochs=1,
)

from transformers import AutoConfig, AutoModelForCausalLM
model_checkpoint = "bert-base-uncased"

config = AutoConfig.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_config(config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show
from transformers import AutoConfig, AutoModelForMaskedLM

model_type = 'bert'


config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

In [ ]:
trainer.train()

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model("CUBERT")